In [16]:
# Save all badges to DynamoDB

import boto3
import pandas as pd
import requests
import os
from datetime import datetime

# Configure AWS credentials
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY')
aws_secret_access_key = os.environ.get('AWS_SECRET_KEY')
aws_region = os.environ.get('AWS_REGION')
aws_table = os.environ.get('AWS_TABLE')

# Create a DynamoDB resource
dynamodb = boto3.resource(
  'dynamodb', 
  aws_access_key_id=aws_access_key_id,                        
  aws_secret_access_key=aws_secret_access_key,
  region_name=aws_region
)

# Configure Twitch credentials
client_id = os.environ.get('TWITCH_CLIENT_ID')
client_secret = os.environ.get('TWITCH_CLIENT_SECRET')
oauth = f"https://id.twitch.tv/oauth2/token?client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials"

# Get the DynamoDB table
table = dynamodb.Table(aws_table)

response = requests.post(oauth)

# Check the response
if response.status_code == 200:
  # Successful request

  # Set your environmental variable
  api_key = response.json()["access_token"]

  # Construct headers with the required information
  headers = {
    'Authorization': f'Bearer {api_key}',
    'Client-Id': client_id
  }

  # API endpoint URL
  url = 'https://api.twitch.tv/helix/chat/badges/global'

  response = requests.get(url, headers=headers)
  badges = response.json()

  df = pd.DataFrame(badges["data"])

  display(df)

  print(badges["data"][0]["versions"][0]["title"])
  print(badges["data"][0])

  for badge in badges["data"]:
    # Put item into the table
    table.put_item(Item={
      'pk': "BADGE", 
      'sk': badge['set_id'], 
      'name': badge['versions'][0]['title'],
      'versions': badge['versions']
    })
else:
  # Handle errors
  print(f"Error: {response.status_code}, {response.text}")

,set_id,versions
0,1979-revolution_1,"[{'id': '1', 'image_url_1x': 'https://static-c..."
1,60-seconds_1,"[{'id': '1', 'image_url_1x': 'https://static-c..."
2,60-seconds_2,"[{'id': '1', 'image_url_1x': 'https://static-c..."
3,60-seconds_3,"[{'id': '1', 'image_url_1x': 'https://static-c..."
4,H1Z1_1,"[{'id': '1', 'image_url_1x': 'https://static-c..."
...,...,...
110,tyranny_1,"[{'id': '1', 'image_url_1x': 'https://static-c..."
111,user-anniversary,"[{'id': '1', 'image_url_1x': 'https://static-c..."
112,vga-champ-2017,"[{'id': '1', 'image_url_1x': 'https://static-c..."
113,vip,"[{'id': '1', 'image_url_1x': 'https://static-c..."


1979 Revolution
{'set_id': '1979-revolution_1', 'versions': [{'id': '1', 'image_url_1x': 'https://static-cdn.jtvnw.net/badges/v1/7833bb6e-d20d-48ff-a58d-67fe827a4f84/1', 'image_url_2x': 'https://static-cdn.jtvnw.net/badges/v1/7833bb6e-d20d-48ff-a58d-67fe827a4f84/2', 'image_url_4x': 'https://static-cdn.jtvnw.net/badges/v1/7833bb6e-d20d-48ff-a58d-67fe827a4f84/3', 'title': '1979 Revolution', 'description': '1979 Revolution', 'click_action': 'visit_url', 'click_url': 'https://www.twitch.tv/directory/game/1979%20Revolution/details'}]}


In [11]:
# Save all emotes to DynamoDB

import boto3
import pandas as pd
import requests
import os
from datetime import datetime

# Configure AWS credentials
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY')
aws_secret_access_key = os.environ.get('AWS_SECRET_KEY')
aws_region = os.environ.get('AWS_REGION')
aws_table = os.environ.get('AWS_TABLE')

# Create a DynamoDB resource
dynamodb = boto3.resource(
  'dynamodb', 
  aws_access_key_id=aws_access_key_id,                        
  aws_secret_access_key=aws_secret_access_key,
  region_name=aws_region
)

# Configure Twitch credentials
client_id = os.environ.get('TWITCH_CLIENT_ID')
client_secret = os.environ.get('TWITCH_CLIENT_SECRET')
oauth = f"https://id.twitch.tv/oauth2/token?client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials"

# Get the DynamoDB table
table = dynamodb.Table(aws_table)

# Make the API request
response = requests.post(oauth)

# Check the response
if response.status_code == 200:
  # Successful request

  # Set your environmental variable
  api_key = response.json()["access_token"]

  # Construct headers with the required information
  headers = {
    'Authorization': f'Bearer {api_key}',
    'Client-Id': client_id
  }

  # API endpoint URL
  url = 'https://api.twitch.tv/helix/chat/emotes/global'

  response = requests.get(url, headers=headers)
  emotes = response.json()

  df = pd.DataFrame(emotes["data"])

  display(df)

  print(emotes["data"][0])

  for emote in emotes["data"]:
    # Put item into the table
    table.put_item(Item={
      'pk': "EMOTE", 
      'sk': emote['id'], 
      'name': emote['name'], 
      'images': emote['images'], 
      'format': emote['format'], 
      'scale': emote['scale'],
      'themes': emote['theme_mode'],
    })
else:
  # Handle errors
  print(f"Error: {response.status_code}, {response.text}")

,id,name,images,format,scale,theme_mode
0,emotesv2_6e599213ae7042c48690a828d74ad8ff,GalaxyUnpacked,{'url_1x': 'https://static-cdn.jtvnw.net/emoti...,"[static, animated]","[1.0, 2.0, 3.0]","[light, dark]"
1,emotesv2_e41e4d6808224f25ae1fb625aa26de63,GoatEmotey,{'url_1x': 'https://static-cdn.jtvnw.net/emoti...,"[static, animated]","[1.0, 2.0, 3.0]","[light, dark]"
2,emotesv2_c1f4899e65cf4f53b2fd98e15733973a,GoldPLZ,{'url_1x': 'https://static-cdn.jtvnw.net/emoti...,"[static, animated]","[1.0, 2.0, 3.0]","[light, dark]"
3,emotesv2_db3385fb0ea54913bf58fa5554edfdf2,ForSigmar,{'url_1x': 'https://static-cdn.jtvnw.net/emoti...,[static],"[1.0, 2.0, 3.0]","[light, dark]"
4,emotesv2_13b6dd7f3a3146ef8dc10f66d8b42a96,TwitchConHYPE,{'url_1x': 'https://static-cdn.jtvnw.net/emoti...,"[static, animated]","[1.0, 2.0, 3.0]","[light, dark]"
...,...,...,...,...,...,...
304,5,:|,{'url_1x': 'https://static-cdn.jtvnw.net/emoti...,[static],"[1.0, 2.0, 3.0]","[light, dark]"
305,4,>(,{'url_1x': 'https://static-cdn.jtvnw.net/emoti...,[static],"[1.0, 2.0, 3.0]","[light, dark]"
306,3,:D,{'url_1x': 'https://static-cdn.jtvnw.net/emoti...,[static],"[1.0, 2.0, 3.0]","[light, dark]"
307,2,:(,{'url_1x': 'https://static-cdn.jtvnw.net/emoti...,[static],"[1.0, 2.0, 3.0]","[light, dark]"


{'id': 'emotesv2_6e599213ae7042c48690a828d74ad8ff', 'name': 'GalaxyUnpacked', 'images': {'url_1x': 'https://static-cdn.jtvnw.net/emoticons/v2/emotesv2_6e599213ae7042c48690a828d74ad8ff/static/light/1.0', 'url_2x': 'https://static-cdn.jtvnw.net/emoticons/v2/emotesv2_6e599213ae7042c48690a828d74ad8ff/static/light/2.0', 'url_4x': 'https://static-cdn.jtvnw.net/emoticons/v2/emotesv2_6e599213ae7042c48690a828d74ad8ff/static/light/3.0'}, 'format': ['static', 'animated'], 'scale': ['1.0', '2.0', '3.0'], 'theme_mode': ['light', 'dark']}


In [3]:
# Save top clips for the week

import boto3
import pandas as pd
import requests
import os
import uuid
from datetime import datetime, timedelta
from decimal import Decimal  # For handling Decimal types in DynamoDB
from urllib.parse import quote

# Configure Twitch credentials
client_id = os.environ.get('TWITCH_CLIENT_ID')
client_secret = os.environ.get('TWITCH_CLIENT_SECRET')
oauth = f"https://id.twitch.tv/oauth2/token?client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials"

# Configure AWS credentials
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY')
aws_secret_access_key = os.environ.get('AWS_SECRET_KEY')
aws_region = os.environ.get('AWS_REGION')
aws_table = os.environ.get('AWS_TABLE')

# Create a DynamoDB resource
dynamodb = boto3.resource(
  'dynamodb', 
  aws_access_key_id=aws_access_key_id,                        
  aws_secret_access_key=aws_secret_access_key,
  region_name=aws_region
)

# Get the DynamoDB table
table = dynamodb.Table(aws_table)

# Make the API request
response = requests.post(oauth)

# Check the response
if response.status_code == 200:
  # Successful request

  # Set your environmental variable
  api_key = response.json()["access_token"]

  # Construct headers with the required information
  headers = {
    'Authorization': f'Bearer {api_key}',
    'Client-Id': client_id
  }

  response = table.query(
    KeyConditionExpression='pk = :pk',
    ExpressionAttributeValues={
      ':pk': "CLIP"
    }
  )

  # Step 2: Iterate through the results and delete each item
  for item in response['Items']:
    table.delete_item(
      Key={
        'pk': item['pk'],
        'sk': item['sk']  # Assuming you have a sort key, adjust this accordingly
      }
    )

  pagination = None
  all_games = list()
  amount = 50

  # API endpoint URL
  url = f'https://api.twitch.tv/helix/games/top?first={amount}'

  response = requests.get(url, headers=headers)
  games = response.json()

  all_games.extend(games["data"])

  pagination = None
  all_clips = list()
  amount = 100

  # Get today's date at 12:00 AM
  today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

  # Get last week's date at 12:00 AM
  last_week = today - timedelta(days=7)

  # Making strings compatible to Twitch's started_at and ended_at query parameters
  today = today.strftime('%Y-%m-%dT%H:%M:%SZ')
  last_week = last_week.strftime('%Y-%m-%dT%H:%M:%SZ')

  for game in games["data"]: 
    # API endpoint URL
    url = f'https://api.twitch.tv/helix/clips?is_featured={True}&started_at={last_week}&ended_at={today}&game_id={game["id"]}&first={amount}'

    print(game["id"])
    
    response = requests.get(url, headers=headers)
    clips = response.json()

    all_clips.extend(clips["data"])

  print(len(all_clips))

  # Sort the list of dictionaries based on 'view_count' field
  sorted_list = sorted(all_clips, key=lambda x: x['view_count'], reverse=True)

  # Get the top 500 elements
  top_100 = sorted_list[:100]

  # Print or use top 500 elements
  print(len(top_100))

  df = pd.DataFrame(top_100)

  display(df)

  # Save all clips to DynamoDB

  for clip in top_100:
    # Define PK and SK
    pk = f'CLIP'
    sk = clip['id']
    
    # Convert clip object into DynamoDB item format
    item = {
      'pk': pk,
      'sk': sk,
      'url': clip['url'],
      'embed_url': clip['embed_url'],
      'broadcaster_id': clip['broadcaster_id'],
      'broadcaster_name': clip['broadcaster_name'],
      'creator_id': clip['creator_id'],
      'creator_name': clip['creator_name'],
      'video_id': clip['video_id'],
      'game_id': clip['game_id'],
      'language': clip['language'],
      'title': clip['title'],
      'view_count': Decimal(clip['view_count']),  # Convert to Decimal for DynamoDB
      'created_at': clip['created_at'],
      'thumbnail_url': clip['thumbnail_url'],
    }
  
    # Put item into DynamoDB table
    table.put_item(Item=item)
else:
  # Handle errors
  print(f"Error: {response.status_code}, {response.text}")

21779
509658
516575
32982
1864374139
29595
33214
1535225577
511224
32399
518203
512710
493887283
766571430
515025
491487
138585
509110
18122
491931
27471
29452
538054672
490359
509659
26936
498566
460630
745589378
55453844
509660
493057
1337444628
513143
498592
27284
512953
490100
1952699919
27546
143106037
209738946
417528
516588
66082
509672
272263131
513181
509663
497431
4660
100


,id,url,embed_url,broadcaster_id,broadcaster_name,creator_id,creator_name,video_id,game_id,language,title,view_count,created_at,thumbnail_url,duration,vod_offset,is_featured
0,AltruisticSnappyLardBCWarrior-wXqmglbExjdOsNdA,https://clips.twitch.tv/AltruisticSnappyLardBC...,https://clips.twitch.tv/embed?clip=AltruisticS...,411377640,Jynxzi,125389599,yoxic,2063003202,460630,en,Sketch,442898,2024-02-15T02:16:52Z,https://clips-media-assets2.twitch.tv/JxgES8F6...,40.5,7729.0,True
1,AgreeableTenderHornetPRChase-v6yp1q5wezjZpy9h,https://clips.twitch.tv/AgreeableTenderHornetP...,https://clips.twitch.tv/embed?clip=AgreeableTe...,138593293,PeachJars,38551761,PkmnLover9000,,509658,en,Looking,251598,2024-02-16T17:32:41Z,https://clips-media-assets2.twitch.tv/bhnKI7-s...,17.1,NaN,True
2,BlithePuzzledCaterpillarJebaited-bVJ0iTm5SxMa826z,https://clips.twitch.tv/BlithePuzzledCaterpill...,https://clips.twitch.tv/embed?clip=BlithePuzzl...,38718052,Alinity,773456053,mizkifegghehe,2063650837,509658,en,very true,194219,2024-02-15T19:21:32Z,https://clips-media-assets2.twitch.tv/o4OLnm1S...,25.3,1028.0,True
3,TriangularDeliciousPrariedogSmoocherZ-D8GiJvc2...,https://clips.twitch.tv/TriangularDeliciousPra...,https://clips.twitch.tv/embed?clip=TriangularD...,49207184,fps_shaka,465099428,_星の,2062316350,766571430,ja,葛葉に処される釈迦ちゃ,119458,2024-02-14T06:24:45Z,https://clips-media-assets2.twitch.tv/lbSg7Q_B...,28.0,7187.0,True
4,SavoryBigCiderDBstyle-XZ91vWrbrUFQQgs0,https://clips.twitch.tv/SavoryBigCiderDBstyle-...,https://clips.twitch.tv/embed?clip=SavoryBigCi...,37522866,CYR,925286813,thanahtos,2064788257,509658,en,peach hints that its cold outside what cyr doe...,89472,2024-02-17T01:25:48Z,https://clips-media-assets2.twitch.tv/IaqT4V2f...,31.0,10339.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,EncouragingAdventurousAubergineRaccAttack-1jjM...,https://clips.twitch.tv/EncouragingAdventurous...,https://clips.twitch.tv/embed?clip=Encouraging...,184198933,IvZel96,184198933,IvZel96,2065442082,518203,,Пушка страшная из немецкой медиалиги,8981,2024-02-17T16:20:54Z,https://clips-media-assets2.twitch.tv/PYpoatY4...,30.0,2077.0,True
96,AuspiciousCorrectDiscJonCarnage-T01XwhSRIXdpydYX,https://clips.twitch.tv/AuspiciousCorrectDiscJ...,https://clips.twitch.tv/embed?clip=AuspiciousC...,38244180,alanzoka,510341191,r0bbzz_,2064576847,509658,pt-br,ALAN na BOLÉIA kkkkk,8933,2024-02-16T19:18:26Z,https://clips-media-assets2.twitch.tv/yuW-OU9N...,47.5,3040.0,True
97,EndearingBlindingManateeDogFace-QxLmTrCaliWeGsqT,https://clips.twitch.tv/EndearingBlindingManat...,https://clips.twitch.tv/embed?clip=EndearingBl...,485855940,lolpacifictw,485855940,lolpacifictw,,21779,zh-tw,忍很久了,8929,2024-02-15T13:25:30Z,https://clips-media-assets2.twitch.tv/hj_TU0MS...,19.0,NaN,True
98,FrigidTriangularSalsifyUnSane-UhhvraSRT-leF6Zp,https://clips.twitch.tv/FrigidTriangularSalsif...,https://clips.twitch.tv/embed?clip=FrigidTrian...,734378088,xoDee,56105739,TheVoidRose,,509658,en,Dee - Red Panda,8875,2024-02-17T09:00:07Z,https://clips-media-assets2.twitch.tv/Qu9al1IY...,5.0,NaN,True


In [ ]:
# Get top live channels

import pandas as pd
import requests
import os
from datetime import datetime

# Configure Twitch credentials
client_id = os.environ.get('TWITCH_CLIENT_ID')
client_secret = os.environ.get('TWITCH_CLIENT_SECRET')
oauth = f"https://id.twitch.tv/oauth2/token?client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials"

# Make the API request
response = requests.post(oauth)

# Check the response
if response.status_code == 200:
  # Successful request

  # Set your environmental variable
  api_key = response.json()["access_token"]

  # Construct headers with the required information
  headers = {"Authorization": f"Bearer {api_key}", "Client-Id": client_id }

  amount = 100        

  # API endpoint URL
  url = f"https://api.twitch.tv/helix/streams?first={amount}"

  response = requests.get(url, headers=headers)
  channels = response.json()

  df = pd.DataFrame(channels["data"])

  display(df)
else:
  # Handle errors
  print(f"Error: {response.status_code}, {response.text}")

In [ ]:
# Get channel data

import pandas as pd
import requests
import os

# Configure Twitch credentials
client_id = os.environ.get('TWITCH_CLIENT_ID')
client_secret = os.environ.get('TWITCH_CLIENT_SECRET')
oauth = f"https://id.twitch.tv/oauth2/token?client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials"

# Make the API request
response = requests.post(oauth)

# Check the response
if response.status_code == 200:
  # Successful request

  # Set your environmental variable
  api_key = response.json()["access_token"]

  # Construct headers with the required information
  headers = {
    'Authorization': f'Bearer {api_key}',
    'Client-Id': client_id
  }

  pagination = None
  all_games = list()
  amount = 50
  user = "xqc"

  # API endpoint URL
  if user.isdigit():
    url = f'https://api.twitch.tv/helix/users?id={user}'
  else:
    url = f'https://api.twitch.tv/helix/users?login={user}'

  response = requests.get(url, headers=headers)
  channel = response.json()

  df = pd.DataFrame(channel["data"])

  display(df)

  login = channel["data"]

  url = f'https://twitchtracker.com/api/channels/summary/{user}'
  
  response = requests.get(url, headers=headers)
  statistics = response.json()

  print(statistics)

  df = pd.DataFrame([statistics])

  display(df)
else:
  # Handle errors
  print(f"Error: {response.status_code}, {response.text}")

In [ ]:
# Get streams based on parameters

import pandas as pd
import requests
import os

# Configure Twitch credentials
client_id = os.environ.get('TWITCH_CLIENT_ID')
client_secret = os.environ.get('TWITCH_CLIENT_SECRET')
oauth = f"https://id.twitch.tv/oauth2/token?client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials"

# Make the API request
response = requests.post(oauth)

# Check the response
if response.status_code == 200:
  # Successful request

  # Set your environmental variable
  api_key = response.json()["access_token"]

  # Construct headers with the required information
  headers = {"Authorization": f"Bearer {api_key}", "Client-Id": client_id }

  pagination = None
  all_streams = list()
  amount = 100
  game = "33214"
  language = "en"
  viewers = 10
  user = "xqc"

  while True: 
    # API endpoint URL
    if pagination is None:
      url = f"https://api.twitch.tv/helix/streams?type=live&first={amount}"
    else:
      url = f"https://api.twitch.tv/helix/streams?type=live&first={amount}&after={pagination}"

    if game:
      url += f"&game_id={game}"
    if language:
      url += f"&language={language}"

    response = requests.get(url, headers=headers)
    streams = response.json()

    all_streams.extend(streams["data"])

    if not (streams["pagination"]) or len(all_streams) >= 500:
      break
    
    pagination = streams["pagination"]["cursor"]

    print(pagination)       

  df = pd.DataFrame(all_streams)

  display(df)
else:
  # Handle errors
  print(f"Error: {response.status_code}, {response.text}")